# Reproducibility: versioning

1. Create two versions of a dataframe (v1, v2)
2. Commit both to dolt as table: `bar` -> get two commit references
3. Run Flow with two different versions of `bar` (parameter to specify)
4. Read the results table `baz` for the two runs given the `run_id`'s

In [25]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

import pandas as pd
from doltpy.core import Dolt
from doltpy.core.write import import_df

dolt = Dolt.init("foo")

df_v1 = pd.DataFrame({"A": [1,1,1], "B": [1,1,1]})
df_v2 = pd.DataFrame({"A": [1,1,1,2,2,2], "B": [1,1,1,2,2,2]})

import_df(dolt, "bar", df_v1.reset_index(), ["index"], "create")
dolt.add("bar")
dolt.commit("Initialize bar")

v1 = list(dolt.log(number="1").keys())[0]

import_df(dolt, "bar", df_v2.reset_index(), ["index"], "update")
dolt.add("bar")
dolt.commit("Add rows to bar")

v2 = list(dolt.log(number="1").keys())[0]

In [10]:
! cat demo.py

import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

import pickle

from metaflow import FlowSpec, step, DoltDT, Parameter
import pandas as pd
from sklearn import tree

class VersioningDemo(FlowSpec):
    bar_version = Parameter('bar-version',  help="Specifc the tag for the input version", required=True)
    @step
    def start(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:
            self.df = dolt.read_table('bar', commit=self.bar_version)

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self, database='foo', branch="master") as dolt:

            df = self.df
            df["B"] = df["B"].map(lambda x: x*2)

            dolt.write_table(table_name='baz', df=df, pks=['index'])

        self.next(self.end)

    @step
    def end(self):
        pass


if __name__ == '__main__':
    VersioningDemo()


In [26]:
!poetry run python3 demo.py run --bar-version $v1

Metaflow 2.2.5.post24+git1f18147 executing VersioningDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 19:02:28.232 Workflow starting (run-id 1610938948223837):
2021-01-17 19:02:28.239 [1610938948223837/start/1 (pid 31582)] Task is starting.
2021-01-17 19:02:29.325 [1610938948223837/start/1 (pid 31582)] 01-17 19:02:29 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 19:02:29.356 [1610938948223837/start/1 (pid 31582)] 01-17 19:02:29 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 19:02:29.417 [1610938948223837/start/1 (pid 31582)] 01-17 19:02:29 doltpy.core.dolt INFO     * master                                        	t15uoa82uaqn2sjq1eq0qtgf04pu275f
2021-01-17 19:02:29.580 [1610938948223837/start/1 (pid 31582)] 
2021-01-17 19:02:29.580 [1610938948223837/start/1 (pid 31582)] 01-17 19:02:29 dolt

2021-01-17 19:02:32.228 [1610938948223837/middle/2 (pid 31616)] 01-17 19:02:32 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-17 19:02:32.410 [1610938948223837/middle/2 (pid 31616)] Task finished successfully.
2021-01-17 19:02:32.417 [1610938948223837/end/3 (pid 31656)] Task is starting.
2021-01-17 19:02:33.596 [1610938948223837/end/3 (pid 31656)] 01-17 19:02:33 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
2021-01-17 19:02:33.603 [1610938948223837/end/3 (pid 31656)] 01-17 19:02:33 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-17 19:02:33.776 [1610938948223837/end/3 (pid 31656)] Task finished successfully.
2021-01-17 19:02:33.776 Done!
01-17 19:02:33 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-17 19:02:33 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [27]:
!cd foo && dolt log -n 1

commit 01pnohoe7p3tevbg6u35k36a4ljobqjb
Author: Max Hoffman <maximilian.wolfgang1@gmail.com>
Date:   Sun Jan 17 19:02:31 -0800 2021

	VersioningDemo/1610938948223837/middle/2



In [28]:
# save for later
run1_id = "1610938948223837"

In [29]:
!poetry run python3 demo.py run --bar-version $v2

Metaflow 2.2.5.post24+git1f18147 executing VersioningDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-17 19:02:45.773 Workflow starting (run-id 1610938965767319):
2021-01-17 19:02:45.781 [1610938965767319/start/1 (pid 31675)] Task is starting.
2021-01-17 19:02:46.964 [1610938965767319/start/1 (pid 31675)] 01-17 19:02:46 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 19:02:46.993 [1610938965767319/start/1 (pid 31675)] 01-17 19:02:46 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-17 19:02:47.054 [1610938965767319/start/1 (pid 31675)] 01-17 19:02:47 doltpy.core.dolt INFO     * master                                        	01pnohoe7p3tevbg6u35k36a4ljobqjb
2021-01-17 19:02:47.226 [1610938965767319/start/1 (pid 31675)] 
2021-01-17 19:02:47.226 [1610938965767319/start/1 (pid 31675)] 01-17 19:02:47 dolt

2021-01-17 19:02:50.034 [1610938965767319/middle/2 (pid 31709)] Task finished successfully.
2021-01-17 19:02:50.042 [1610938965767319/end/3 (pid 31749)] Task is starting.
2021-01-17 19:02:51.212 [1610938965767319/end/3 (pid 31749)] 01-17 19:02:51 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
2021-01-17 19:02:51.219 [1610938965767319/end/3 (pid 31749)] 01-17 19:02:51 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-17 19:02:51.378 [1610938965767319/end/3 (pid 31749)] Task finished successfully.
2021-01-17 19:02:51.378 Done!
01-17 19:02:51 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-17 19:02:51 doltpy.core.system_helpers INFO     No processes to clean up, exiting


In [30]:
!cd foo && dolt log -n 1

commit ttd37vr2p5h9j6ormidbn0u7ojubbetg
Author: Max Hoffman <maximilian.wolfgang1@gmail.com>
Date:   Sun Jan 17 19:02:49 -0800 2021

	VersioningDemo/1610938965767319/middle/2



In [31]:
run2_id = "1610938965767319"

In [32]:
from metaflow.datatools.dolt import DoltDT, DoltRun
d1 = DoltRun(flow_name="VersioningDemo", run_id=run1_id)
d2 = DoltRun(flow_name="VersioningDemo", run_id=run2_id)

res1 = d1.writes[0].data
res2 = d2.writes[0].data

foo 01pnohoe7p3tevbg6u35k36a4ljobqjb baz
foo ttd37vr2p5h9j6ormidbn0u7ojubbetg baz


In [33]:
res1

,index,A,B
0,0,1,11
1,1,1,11
2,2,1,11


In [34]:
res2

,index,A,B
0,0,1,11
1,1,1,11
2,2,1,11
3,3,2,22
4,4,2,22
5,5,2,22
